In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import time


In [14]:

# ---------------------------------------------------------------------------------------------- Configuration ---
# --- Input --- 
img_path = 'input/yurimaguas_ls7_2001181_lrg.jpg' 
# --- Output --- 
output_path = 'output/output_brightness_sorted_image4.png'
output_segmented_path = 'output/output_kmeans_segmented4.png'
output_sorted_path = 'output/output_cluster_brightness_sorted4.png'
# --- Diagnostic 
# --- k-means testing range // code commented out below
k_range = range (2,11)
output_elbow_plot_path = 'output_elbow_plot.png'
output_silhouette_plot_path = 'output_silhouette_plot.png'
# --- K-means settings
CHOSEN_K = 6


In [15]:
# --------------------------------------------------------------------------------------- Load Image + convert To numpy Array---
# Read image - OpenCV - BGR order [b,g,r] - cv2.IMREAD_UNCHANGED preserve alpha
#
img_bgr = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
# Check if image exists 
if img_bgr is None:
    print(f"Error: Could not load image from {img_path}")
    # if no image exit 
    exit()
# Shape of Image 
if len(img_bgr.shape) != 3 or img_bgr.shape[2] < 3:
    print(f"Error: Image does not appear to be a standard 3-channel color image. Shape: {img_bgr.shape}")
    exit()
# Make a copy to avoid modifying the original array if needed elsewhere
img_original_shape = img_bgr.shape
height, width, channels = img_original_shape
num_pixels = height * width
print(height)
print(f"The width is {width}")
print(num_pixels)
#
# flatten img to 2d array with columns = channels (3) tuple 
pixel_data_flat = img_bgr.reshape(-1, channels).astype(np.float32)
#

1701
The width is 1913
3254013


In [16]:
print(f"Running K-Means with k={CHOSEN_K}...")
start_time_final_kmeans = time.time()
final_kmeans = KMeans(n_clusters=CHOSEN_K, random_state=42, n_init=10)
final_kmeans.fit(pixel_data_flat)
cluster_labels = final_kmeans.labels_
cluster_centers = final_kmeans.cluster_centers_ # These are the average colors for each cluster
end_time_final_kmeans = time.time()
print(f"Final K-Means took {end_time_final_kmeans - start_time_final_kmeans:.2f} seconds.")

Running K-Means with k=6...
Final K-Means took 7.16 seconds.


In [18]:
# ---------------------------------------------------------------------------------- Create Segmented Image from copy of original usking kmeans cluster labels ---
# Replace each pixel's value with its corresponding cluster center color
segmented_pixel_data = cluster_centers[cluster_labels]
segmented_image = segmented_pixel_data.reshape(img_original_shape)

# Convert back to original data type for saving (or normalize to uint8)
if img_bgr.dtype == np.uint16:
    segmented_image_save = segmented_image.astype(np.uint16)
    # Or normalize for PNG ---- segmented_image_save = cv2.normalize(segmented_image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
elif img_bgr.dtype == np.uint8:
    segmented_image_save = segmented_image.astype(np.uint8)
else:
    segmented_image_save = segmented_image # Hope for the best

save_success_seg = cv2.imwrite(output_segmented_path, segmented_image_save)

if save_success_seg:
    print(f"K-Means segmented image saved successfully as: {output_segmented_path}")
else:
    print(f"Error: Failed to save segmented image to {output_segmented_path}")




K-Means segmented image saved successfully as: output/output_kmeans_segmented4.png


In [19]:

# ----------------------------------------------------------------------------------------------------------- Calculate Brightness (Luminance) ---
# prolly wanna do greyscale for this in the future(?)
img_float = img_bgr.astype(np.float32)
brightness = 0.114 * img_float[:,:,0] + 0.587 * img_float[:,:,1] + 0.299 * img_float[:,:,2]
all_brightness_scores = brightness.flatten()

# --- Sort Pixels by Cluster, then Brightness ---
print("Sorting pixels by Cluster ID, then by Brightness...")
all_original_pixels = img_bgr.reshape(-1, channels)

# Combine the data needed for sorting: (original_pixel, cluster_label, brightness) (brigtness currently removed )
#  indices reconstruct sorted pixel array 
pixel_indices = np.arange(num_pixels)
combined_sort_data = list(zip(pixel_indices, cluster_labels))

# Sort first by cluster label (element 1),  ////////////////////// (removed temp) then by brightness (element 2)
# Ascending for both: cluster 0 first, darkest within cluster first
sorted_combined_data = sorted(combined_sort_data, key=lambda item: (item[1]))

# Get the original pixel values in the new sorted order
sorted_indices_final = [item[0] for item in sorted_combined_data]
final_sorted_pixels_array = all_original_pixels[sorted_indices_final]

# Create pairs of (cluster_label, brightness_score) for sorting criteria
# np.lexsort for efficient multi-key sorting.
# It sorts by the last column first, so we provide brightness then cluster label.
sort_keys = (all_brightness_scores, cluster_labels)
sorted_order_indices = np.lexsort(sort_keys)

sorted_cluster_labels = cluster_labels[sorted_order_indices]
final_sorted_pixels_array = cluster_centers[sorted_cluster_labels] # <-- THE KEY CHANGE
# --- Place Pixels Sequentially ---
# Reshape the array of centroid colors (which is already in the final sorted order)
output_image_sorted = final_sorted_pixels_array.reshape(img_original_shape)


# --- Place Pixels Sequentially ---
# output_image_sorted = final_sorted_pixels_array.reshape(img_original_shape)


Sorting pixels by Cluster ID, then by Brightness...


In [20]:
# --- Save Final Sorted Output Image ---
if img_bgr.dtype == np.uint16:
    output_image_sorted_save = cv2.normalize(output_image_sorted, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint16)
elif img_bgr.dtype == np.uint8:
    output_image_sorted_save = output_image_sorted.astype(np.uint8)
else:
    output_image_sorted_save = output_image_sorted

save_success_sort = cv2.imwrite(output_sorted_path, output_image_sorted_save)
if save_success_sort:
    print(f"Cluster+Brightness sorted image saved successfully as: {output_sorted_path}")
else:
    print(f"Error: Failed to save sorted image to {output_sorted_path}")

print("\nProcess Complete.")

Cluster+Brightness sorted image saved successfully as: output/output_cluster_brightness_sorted4.png

Process Complete.
